In [ ]:
# ToDo:

# Have working on Colab ON HOLD

# General usability. PROGRESS

# General form layout imprvements. PROGRESS

# Make "Here is the text: " hardcoded at the end of the users labelling text/

# Make sure the instructions are clear and concise and make sense given the updates PROGRESS

# Sort the placeholder ordering to reflect the order in the template

# Put all background code in a DataGenerator class

# General code quality
# - probably some repeated code between forms that could be refactored
# - much of the form code stitched together from LLM generated code for fast prototyping
# --- Is it all necessary?
# --- Can it be improved?
# --- Does it make sense?
# TODO Aayush to Matthew: It makes sense to do this, though I'd say after we get the usability stuff done? (forms, usability improvements, directory of labels etc.). Since this is mainly a backend improvement, and the notebook users won't care.

# Don't allow saving as .csv.json in labelling form DONE (solved using UX)

# Track cost of label generation

# Prompt novelty
# --- random seed but record seeds
# --- length
# --- Sample from top 5?
# --- higher shot in k-shot
# --- bigger model - more expensive

# Pick up from where we left off of API call fails
# Do you loose everything if you stop the API call?
# TODO: Aayush to Matthew: How important of a problem is this to solve?
# MAtthew to Aayush: Not very important.





# DONE:

# Directory of all pre-existing labels - biggest change
# Backslashes necessary in labelling prompts?
# Form for initial stuff like model, temperature etc?

# Notebook explanation

The purpose of this notebook is to use a prompt given to some LLM ("LLM A") to generate a dataset of different prompts to be fed to some other LLM ("LLM B" - potentially the same as "LLM A") for which you have access to the weights. This dataset of prompts will be used to create a "representation vector" of a property or concept for LLM B in order to "steer" LLM B to have more of that property or concept in its outputs. This Notebook does not cover the creation of representations or steering, only the the creation of a dataset of prompts.

The kinds of prompts you want to generate should be about the property or concept you want to steer the LLM towards, not necessary literally mentioning it - e.g. prompts about politeness do not necessarily need to have the word polite in them. Part of the point of dataset creation is to explore which kinds of generated prompts yield good representations.

The Notebook works as follows:

- The Setup section loads Python libraries needed to run the code. You do not need to change anything here.
- The Inputs section is where you define the prompt you will use to generate your dataset of prompts. Instructions on how to do this are given. This is the only section of the notebook where you will need to change anything.
- The Review section then generates a small example dataset of prompts and shows them to you. If you like them, continue on to the end of the Notebook.
- If you do not like them, please go back to the Inputs section to refine your prompt to generate your dataset of prompts.
- The Dataset Generation section completes the dataset generation.
- The View Dataset section loads your generated dataset for inspection.
- Your dataset will be stored in /data/inputs/name_of_your_dataset/dataset if you want to use it later.

The datasets will be generated in CSV format and should have the following form, where the first line is the column headings and subsequent lines are for example prompts. The columns ethical_area and ethical_valence are two different labels (classifications) of the prompt. The Notebook will help you generate these labels. This example is for "politeness" prompts:

```
prompt, ethical_area, ethical_valence
"Would you be so kind as to pass the water please.", "polite", 1
"Give me the water now.", "impolite", 0
```

If this is too restrictive, you will be able to create other columns and the notebook will ask you about that too.

Note that where we have text, such as for the prompt or ethical_area, we want it enclosed in quote marks. Lone numbers do not need this.

If there is something unusual you want to do that the current notebook does not permist, please ask, or feel free to try adding code for it. 

# Setup (just run)

In [1]:
import csv
import json
import math
import os
import re
import time
from IPython import get_ipython
from IPython.display import display, HTML, clear_output
from ipywidgets import (
    Button,
    Checkbox,
    Dropdown,
    FloatText,
    HBox,
    IntText,
    Label,
    Layout,
    Output,
    SelectMultiple,
    Text,
    Textarea,
    VBox,
    interactive,
    interact,
    interact_manual,
    fixed,
    widgets,
)
from jinja2 import Environment, FileSystemLoader, meta
from hydra import initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.experimental import compose
from omegaconf import DictConfig, OmegaConf
from openai import OpenAI
import pandas as pd
import yaml

# Basic Settings

The form below two cells below should allow you to enter your open AI API key on the form, or it will find it in your environment variables if you set it.

However, on some machines the form did not allow users to enter their open AI API key. If you did not set your OpenAI key in your environment variables, and you are having trouble entering you open AI API key on the form, enter it in the cell below first and the for, should load it automatically.

Please do not commit and push code with live API keys.

In [2]:
# You only need to enter your API key if you are having issues with the form.
# Uncomment the line (remove the # symbol) below and replace 'your_openai_api_key' with your actual key.

# os.environ['OPENAI_API_KEY'] = 'your_openai_api_key'

In [3]:
# OpenAI API Key
api_key_input = widgets.Text(
    value=os.environ.get("OPENAI_API_KEY", ""),
    description='Enter your OpenAI API key:',
    disabled=False,
    style={'description_width': 'initial'}
)

def save_api_key(api_key):
    os.environ["OPENAI_API_KEY"] = api_key
    print("API key saved to environment variables.")

api_key_button = widgets.Button(description="Save API Key")
api_key_button.on_click(lambda _: save_api_key(api_key_input.value))

api_key_box = widgets.VBox([api_key_input, api_key_button])

# Model Selection
model_options = ['gpt-4-0125-preview', 'gpt-4', 'gpt-3.5-turbo']
model_dropdown = widgets.Dropdown(
    options=model_options,
    value='gpt-4',
    description='OpenAI Model:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Temperature
temperature_input = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=2.0,
    step=0.1,
    description='Temperature:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    style={'description_width': 'initial'}
)

# Filename
filename_input = widgets.Text(
    value='justice',
    description='Filename (without extension):',
    disabled=False,
    style={'description_width': 'initial'}
)

# Total Number of Examples
total_examples_input = widgets.IntText(
    value=100,
    description='Total Number of Examples:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Number of Examples per Request
examples_per_request_input = widgets.IntText(
    value=100,
    description='Examples per Request:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Display the widgets
display(api_key_box)
display(model_dropdown)
display(temperature_input)
display(filename_input)
display(total_examples_input)
display(examples_per_request_input)

# Retrieve the values
def get_inputs():
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    model = model_dropdown.value
    temperature = temperature_input.value
    filestem = filename_input.value
    total_num_examples = total_examples_input.value
    num_examples_per_request = examples_per_request_input.value
    return client, model, temperature, filestem, total_num_examples, num_examples_per_request

# Reset the values to default
def reset_values():
    model_dropdown.value = 'gpt-4'
    temperature_input.value = 1.0
    filename_input.value = 'justice'
    total_examples_input.value = 100
    examples_per_request_input.value = total_examples_input.value
    print("Values reset to default.")

def on_submit_click(button):
    client, model, temperature, filestem, total_num_examples, num_examples_per_request = get_inputs()
    display_inputs(client, model, temperature, filestem, total_num_examples, num_examples_per_request)

def display_inputs(client, model, temperature, filestem, total_num_examples, num_examples_per_request):
    print("Entered information:")
    print(f"Client: {client}")
    print(f"OpenAI Model: {model}")
    print(f"Temperature: {temperature}")
    print(f"Filename: {filestem}.csv")
    print(f"Total Number of Examples: {total_num_examples}")
    print(f"Examples per Request: {num_examples_per_request}")

submit_button = widgets.Button(description="Submit")
submit_button.on_click(on_submit_click)  # Update this line

reset_button = widgets.Button(description="Reset")
reset_button.on_click(lambda _: reset_values())

button_box = widgets.HBox([submit_button, reset_button])
display(button_box)

Dropdown(description='OpenAI Model:', index=1, options=('gpt-4-0125-preview', 'gpt-4', 'gpt-3.5-turbo'), style…

FloatSlider(value=1.0, continuous_update=False, description='Temperature:', max=2.0, readout_format='.1f', sty…

Text(value='justice', description='Filename (without extension):', style=TextStyle(description_width='initial'…

IntText(value=100, description='Total Number of Examples:', style=DescriptionStyle(description_width='initial'…

IntText(value=100, description='Examples per Request:', style=DescriptionStyle(description_width='initial'))

API key saved to environment variables.
Entered information:
Client: <openai.OpenAI object at 0x7febb0b4a8f0>
OpenAI Model: gpt-4
Temperature: 1.0
Filename: terror.csv
Total Number of Examples: 100
Examples per Request: 100


In [4]:
client, model, temperature, filestem, total_num_examples, num_examples_per_request = get_inputs()
display_inputs(client, model, temperature, filestem, total_num_examples, num_examples_per_request)

Entered information:
Client: <openai.OpenAI object at 0x7febb0b496f0>
OpenAI Model: gpt-4
Temperature: 1.0
Filename: terror.csv
Total Number of Examples: 100
Examples per Request: 100


## Hints

**OpenAI Api Key**
- Get an API key [here](https://auth0.openai.com/u/signup/identifier?state=hKFo2SA1REJ3dGFZVTllNHFvYUFkY2RrWEJpUUVMUWxvel91VqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIFg1Z2NKOU9hUk4yYUFmWGxyTHlscmtNTmMxbDF5dWZTo2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q).
- You will need to put some token amount of money on to use gpt-4-0125-preview. It's something like $1. Might be $10.

**OpenAI Model**
- See [here](https://platform.openai.com/docs/models/) for the list of models: 

**Temperature**
- Lower values for temperature result in more consistent outputs (e.g. 0.2), while higher values generate more diverse and creative results (e.g. 1.0)
- If your generated sentences are too similar/boring, try increasing the number. Go the other way if too wacky.
- More details [here](https://platform.openai.com/docs/guides/text-generation/how-should-i-set-the-temperature-parameter)

**Filename**
- Saves the dataset to a file with this name.
- Try to give it a specific name e.g. "honesty_v2.csv" or "honesty_pairs_v2.csv", something that will help you remember what it is especially if you are experimenting with variations.
- Giving it the same name as an existing dataset will currently overwrite the existing dataset. TODO: Resolve this.

**Total Number of Examples**
- This is the total number of prompts you want to generate in your dataset of prompts.
- If you want to generate x prompts in total, put x here.
- What is the max number of examples one should put here?

**Number of Examples per request**
- TODO: @Matthew should we get rid of this variable since Rasmus wants everything in one batch, or is there still usefulness to it?
- This is the number of prompt examples that are generated per request to the model.
- The reason we have this is because the LLM has a limit on the number of words it can process in one request. If you put a number that is too high here, you will not get the number of prompts you want.




TODO: Kept this text just in case for now.

Enter below the number of examples per request. This is the number of prompt examples you want to generate in one request to the LLM.

- This is different from the total_num_examples variable you entered above.
- The reason we have this is because the LLM has a limit on the number of tokens it can process in one request. If you put a number that is too high here, you will not get the number of prompts you want.
- The number of tokens for gpt-4-0125-preview is 128,000 based on details given here: https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo.
- From a quick internet search, I think this model uses about 1.3 tokens per word. So you might have about 4096/1.3 = 98,461 words to play with if using this model. Remember, the number of tokens or words includes those in the prompt.
- This should be more than enough to generate your entire dataset in one go, so you could try setting num_examples_per_request to the same number as total_num_examples. One reason to err on slightly fewer examples is that bigger responses from OpenAI take longer to generate. If you lose your internet connection before generation has ended, you lose your entire generated dataset. If you generate the data in small batches, you will have a partially complete dataset up until the batch that fails.
- This Notebook will make sure that the total number of examples you want to generate as defined in total_num_examples are generated, but it will make multiple requests to the LLM. Eg if you made total_num_examples = 100 and num_examples_per_request = 5, then this code will automatically make 100/5 = 20 requests to the LLM to generate the 100 examples you want.


# Template Creation


Example Template:
> *Please generate a sentence that embodies the concept of {{concept}}, or the opposite of {{concept}} in the context of {{context}}, choosing either with 50% probability. Only write this sentence and nothing else.*

What is a template?
- A generic prompt stucture.
- Any variable/placeholder you might want to pass to your template should be include within a pair of braces like so: {{ your_variable_name }}
- You want to use placeholders where you might want to switch out different parts of the prompt. E.g. you might write {{concept}} instead of politeness or justice or what have you so that you can decide later what to add.
- You will be given the opportunity to fill in placeholders at the end of this form.
- You should write your templates as if you expect only to get one entry of yoiur dataset back. E.g. as if you are asking for one sentence about a concept, not however many you want your dataset to contain. The code will handle creating a dataset for you based on the number you added earlier for dataset size.

In [5]:
display(HTML('''
<style>
    .widget-label {
        white-space: normal;
        word-wrap: break-word;
        overflow-wrap: break-word;
    }
</style>
'''))

In [6]:
# Set up Jinja environment
template_dir = '../data/inputs/templates'
output_dir = '../data/inputs/prompts'
env = Environment(loader=FileSystemLoader(template_dir))

def load_templates():
    """Load template files from the template directory."""
    return [os.path.splitext(f)[0] for f in os.listdir(template_dir) if f.endswith('.j2')]

def create_template_dropdown():
    """Create a dropdown widget for selecting templates."""
    templates = load_templates()
    default_template = 'blank_template'
    return Dropdown(options=templates, value=default_template if default_template in templates else None)

def create_template_content_input():
    """Create a textarea widget for editing template content."""
    return Textarea(rows=10)

def create_filename_input():
    """Create a text input widget for entering a new template filename."""
    return Text(value='new_template')

def create_save_button():
    """Create a button widget for saving the template."""
    button = Button(description='Save')
    button.on_click(save_template)
    return button

def create_use_button():
    """Create a button widget for using the template."""
    button = Button(description='Use Template')
    button.on_click(use_template)
    return button

In [7]:
def save_template(button):
    """Save the template content to a file."""

    # Replace any file extension from the text, if existent, with .j2
    new_filename = filename_input.value.split('.')[0] + '.j2'
    new_template_path = os.path.join(template_dir, new_filename)
    if new_filename == "blank_template.j2":
        with save_warning_output:
            print('Cannot overwrite the "blank_template.j2" file.')
    elif os.path.exists(new_template_path):
        with save_warning_output:
            print(f'File "{new_filename}" already exists. Do you want to overwrite it?')
            overwrite_button = Button(description='Overwrite')
            overwrite_button.on_click(lambda _: save_template_content(new_template_path, True))
            cancel_button = Button(description='Cancel')
            cancel_button.on_click(lambda _: save_warning_output.clear_output())
            display(overwrite_button, cancel_button)
    else:
        save_template_content(new_template_path)

def save_template_content(file_path, overwrite=False):
    """Save the template content to the specified file path."""

    template_content = template_content_input.value
    with open(file_path, 'w') as f:
        f.write(template_content)
    with success_output:
        clear_output()
        print(f'Template saved as "{os.path.basename(file_path)}".')
    
    if not overwrite:
        # Refresh the template dropdown
        template_dropdown.options = load_templates()

        new_template_name = os.path.basename(file_path).split('.')[0]
        template_dropdown.value = new_template_name

def load_template_content(change):
    """Load the content of the selected template into the template content input."""

    template_name = change['new'] + '.j2'
    template_path = os.path.join(template_dir, template_name)
    with open(template_path, 'r') as f:
        template_content = f.read()
    template_content_input.value = template_content

def render_template(template_name, user_input):
    """Render the template with the provided user input."""
    template = env.get_template(template_name)
    return template.render(**user_input)

def use_template(button):
    """Use the selected template and prompt the user to fill in the
    template variables."""

    template_name = template_dropdown.value + '.j2'
    template_source = env.loader.get_source(env, template_name)[0]
    parsed_content = env.parse(template_source)
    variables = meta.find_undeclared_variables(parsed_content)

    load_form = VBox()
    placeholders = {}
    for var in variables:
        placeholder_input = Text(
            description=f'Enter value for "{var}":',
            layout=Layout(width='auto', min_width='200px'),
            style={'description_width': 'initial'}
        )
        load_form.children += (placeholder_input,)
        placeholders[var] = placeholder_input
        
    output_filename_input = Text(
        description='Enter a filename to save the template with these values (e.g. my_prompt):',
        layout=Layout(width='auto', min_width='200px'),
        style={'description_width': 'initial'})
    load_form.children += (output_filename_input,)

    render_button = Button(description='Save and Render')
    render_warning_output = Output()
    
    warning_and_buttons_layout = VBox([
        render_button,
        render_warning_output
    ])
    
    load_form.children += (warning_and_buttons_layout,)

    # Apply CSS styling to the form container
    load_form.layout.width = '100%'
    load_form.layout.min_width = '400px'
    load_form.add_class('my-form')
    
    display(load_form)

    def on_render_and_save(button):
        """Render the template with the provided user input and globally save the rendered text."""

        rendered_text = render_and_save(button)
        if rendered_text is not None:
            # Assign the rendered_text to a variable in the global scope using globals()
            globals()['rendered_prompt'] = rendered_text
    
    def render_and_save(button):
        """Render the template with the provided user input and save the rendered text to a file."""

        placeholder_values = {var: widget.value for var, widget in placeholders.items()}
        rendered_text = render_template(template_name, placeholder_values)
        output_filename = output_filename_input.value
        
        # Remove any existing file extension from the output filename
        output_filename = os.path.splitext(output_filename)[0] + '.txt'
                
        output_path = os.path.join(output_dir, output_filename)
        
        if os.path.exists(output_path):
            with render_warning_output:
                print(f'File "{output_filename}" already exists. Do you want to overwrite it?')
                overwrite_button = Button(description='Overwrite')
                overwrite_button.on_click(lambda _: save_rendered_content(output_path, rendered_text, True))
                cancel_button = Button(description='Cancel')
                cancel_button.on_click(lambda _: render_warning_output.clear_output())
                display(overwrite_button, cancel_button)
        else:
            save_rendered_content(output_path, rendered_text)
        
        return rendered_text

    render_button.on_click(on_render_and_save)

def save_rendered_content(file_path, rendered_text, overwrite=False):
    """Save the rendered text to the specified file path."""

    with open(file_path, 'w') as f:
        f.write(rendered_text)
    with success_output:
        print(f'Rendered text saved as "{os.path.basename(file_path)}".')
    
    # Update the preview output widget with the rendered template
    with preview_output:
        print('Template Preview:')
        print(rendered_text)

In [8]:
def create_widgets():
    """Create the widgets used in the form."""

    global template_dropdown, template_content_input, filename_input, save_button, use_button, preview_button
    global save_warning_output, success_output, preview_output

    template_dropdown = create_template_dropdown()
    template_content_input = create_template_content_input()
    filename_input = create_filename_input()
    save_button = create_save_button()
    use_button = create_use_button()
    save_warning_output = Output()
    success_output = Output()
    preview_output = Output()

def create_layout():
    """Create the main layout of the form."""
    
    input_layout = VBox([
        Label(value='Template Manager', style={'font_weight': 'bold', 'font_size': '18px'}),    
        Label(value='Create a new template, or select/modify an existing one.'),
        template_dropdown,
        Label(value='Edit the template content:'),
        template_content_input,
        Label(value="If you have created a new template or modified an existing one, enter a name for it here. Skip this step if you're directly using an existing template without modifying it."),
        filename_input,
    ])
    
    save_button_layout = HBox([save_button])
    
    warning_and_success_layout = VBox([
        save_warning_output,
        success_output
    ])
    
    separator_layout = HBox([Label(value='─' * 50, style={'font_size': '20px'})])
    
    use_button_description = Label(value='''Once you have decided on a template, let's use it. Once you press the button below, you'll be asked to fill in the relevant variables.''',
                                        layout=Layout(width='auto'))
    
    use_button_layout = HBox([use_button])
    
    button_layout = VBox([
        save_button_layout,
        warning_and_success_layout,
        separator_layout,
        use_button_description,
        use_button_layout
    ], layout=Layout(margin='20px 0px'))
    
    output_layout = VBox([
        preview_output,
    ])
    
    main_layout = VBox([input_layout, button_layout, output_layout], layout=Layout(width='auto'))
    return main_layout

def initialize():
    create_widgets()
    template_dropdown.observe(load_template_content, names='value')
    layout = create_layout()
    display(layout)

In [9]:
initialize()

Running the code below allows you to check you are happy with your prompt. If not, reload the form and fix it.

In [10]:
print("Rendered Prompt:")
print(rendered_prompt)

Rendered Prompt:
Please generate a sentence that embodies the concept of terror, or the opposite of terror in the context of dreams, choosing either with 50% probability. Only write this sentence and nothing else.


Enter below your prompts for generating dataset labels.

Instructions

- The previous form allows you to generate a dataset of prompts.
- The next form allows you to create questions that will label each prompt from the generated set of prompts.
- Each generated prompt is sent one by one with the question back through the LLM as a new combined prompt to get the label.
- Call a prompt from your generated dataset of prompts P.
- Imagine the generated dataset of prompts are either polite or impolite statements e.g.
- "Would you be so kind as to pass the water please." or "Give me the water now."
- You want to label each prompt as "polite" or "impolite".
- In the following form you will be asked to come up with pairs of "headings" and "labellings"
- A "heading" is the name for the column for the labelling in your dataset.
- A "labelling" is the question you will be asking to label the generated prompt.
- In the example below:


```
prompt, ethical_area, ethical_valence
"Would you be so kind as to pass the water please.", "polite", 1
"Give me the water now.", "impolite", 0
```

- The headings are "ethical_area" and "ethical_valance"
- The labellings that got us the labels might be:
- "Do you think the start of the response in the following text is polite or impolite? Output only the word "polite" for polite, or "impolite" for impolite in single word response within single quote marks. Here is the text: "
- "Do you think the start of the response in the following text is polite or impolite? Output only the number 1 for polite, or 0 for impolite. Here is the text: "
- Call a specific heading H and a specific labelling L.
- P and L would be passed back to the LLM as L + P e.g.
- "Do you think the start of the response in the following text is polite or impolite? Output only the word "polite" for polite, or "impolite" for impolite in single word response within single quote marks. Here is the text: Give me the water now."


Details

- You may need to experiment a little to get good labelling from the LLM.
- It is not certain that the LLM will be able to perform the labelling you are asking for, and if it can it may make mistakes.
- You need to check the labels are correct both in the Review section and after the full Dataset Generation section.
- You need to generate labels for ethical_area and ethical_valency at a minimum for the way the code currently works.
- In the example above, ethical_area always map to the same ethical_valency and vice versa. In other words they are synomyms. They need not be though.
- Your label's name and the prompt to generate it should be enclosed in quote marks. 
- It is a good idea to ask the LLM for the single word or number or whatever else you want so it does not add in unnecessary additional comments such as "Sure, this sentence is impolite" etc which we do not want as part of our label.

In [29]:
class HeadingLabellingForm(widgets.VBox):
    def __init__(self):
        super().__init__()
        self.hl_pairs = {}
        self.hl_rows = []  # You may not need this as a list anymore if you opt for a dictionary approach.
        self.hl_row_dict = {}  # New dictionary for row widgets
        self.next_row_id = 0  # Counter for unique row IDs

        self.output_dir = '../data/inputs/labels'

        self.top_text = widgets.HTML(value="""
            <h3>Heading-Labelling Form</h3>
            <p>Use this form to load, modify or create heading-labelling schemes.</p>
            <p>1. To load an existing scheme, select the scheme from the dropdown menu.</p>
            <p>2. To modify the loaded scheme, add or remove heading-labelling pairs using the buttons below.</p>
            <p>3. To create a new scheme, add pairs using the "Add Pair" button withoout loading an existing scheme.</p>
            <p>4. You can also add any existing heading-labelling pairs one by one from all existing schemes combined using the "Add Existing Pair" button.</p>
            <p>5. Click the "Finish" button to log the heading-labellings.</p>
            <p>6. Enter a name for the scheme and click "Save" to save it.</p>
        """)

        # Create a Dropdown widget with a specified layout
        # Adjust the width of the dropdown to take the remaining space
        self.hl_dropdown = widgets.Dropdown(
            options=self.load_hl_files(),
            description='Select a Scheme:',
            layout=Layout(width='auto')  # Set width to auto to take up the available space
        )
        self.hl_dropdown.style.description_width = 'initial'

        # Wrap the dropdown in an HBox that fills the horizontal space
        self.hl_dropdown_container = widgets.HBox(
            [self.hl_dropdown],
            layout=Layout(flex='1', display='flex', width='100%')  # The flex property will allow it to grow
        )
        self.hl_dropdown.observe(self.load_hl_file, names='value')

        self.existing_pairs_dropdown = widgets.Dropdown(options=self.load_all_hl_pairs(), description='Existing Pairs:')

        self.add_button = widgets.Button(description='Add Pair')
        self.add_button.on_click(self.add_hl_pair)

        self.add_existing_button = widgets.Button(description='Add Existing Pair')
        self.add_existing_button.on_click(self.add_existing_pair)

        self.finish_button = widgets.Button(description='Finish')
        self.finish_button.on_click(self.finish_headings)

        self.output = widgets.Output()

        self.filename_input = widgets.Text(placeholder='Enter a name for this scheme')

        self.save_button = widgets.Button(description='Save')
        self.save_button.on_click(self.save_dictionary)

        self.children = [
            self.top_text,
            self.hl_dropdown,
            widgets.HBox([self.add_button, self.existing_pairs_dropdown, self.add_existing_button]),
            *self.hl_rows,
            self.finish_button,
            self.output,
            self.filename_input,
            self.save_button,
        ]

    def load_hl_files(self):
        return [''] + [os.path.splitext(f)[0] for f in os.listdir(self.output_dir) if f.endswith('.json')]

    def load_hl_file(self, change):
        if change['new']:
            hl_filename = change['new'] + '.json'
            hl_file_path = os.path.join(self.output_dir, hl_filename)
            with open(hl_file_path, 'r') as file:
                self.hl_pairs = json.load(file)
            self.update_hl_rows()
        else:
            self.hl_pairs = {}
            self.update_hl_rows()

    def add_hl_pair(self, button, heading='', labelling=''):
        heading_input = widgets.Text(value=heading, placeholder='Enter heading key')
        labelling_input = widgets.Text(value=labelling, placeholder='Enter labelling value')
        remove_button = widgets.Button(description='Remove')

        # Assign a unique ID to the row and increment the counter
        row_id = self.next_row_id
        self.next_row_id += 1

        # Create the row with the heading, labelling inputs, and remove button
        row = widgets.HBox([heading_input, labelling_input, remove_button])
        
        # Store the row in the dictionary with its unique ID
        self.hl_row_dict[row_id] = row

        # Assign the row ID to the button for later reference
        remove_button.row_id = row_id
        
        # Set up the button's on_click event
        remove_button.on_click(self.remove_hl_pair)

        # Update the form layout to include the new row
        self.update_form_layout()


    def add_hl_pair(self, button, heading='', labelling=''):
        heading_input = widgets.Text(value=heading, placeholder='Enter heading key')
        # Specify a larger width for the labelling input
        labelling_input = widgets.Text(value=labelling, placeholder='Enter labelling value', layout=Layout(flex='1'))

        remove_button = widgets.Button(description='Remove')

        # Assign a unique ID to the row and increment the counter
        row_id = self.next_row_id
        self.next_row_id += 1

        # Create the row with the heading, labelling inputs, and remove button
        row = widgets.HBox([heading_input, labelling_input, remove_button])

        # Store the row in the dictionary with its unique ID
        self.hl_row_dict[row_id] = row

        # Assign the row ID to the button for later reference
        remove_button.row_id = row_id
        
        # Set up the button's on_click event
        remove_button.on_click(self.remove_hl_pair)

        # Update the form layout to include the new row
        self.update_form_layout()

    def add_existing_pair(self, button):
        if self.existing_pairs_dropdown.value:
            self.add_hl_pair(None, self.existing_pairs_dropdown.value[0], self.existing_pairs_dropdown.value[1])

    def remove_hl_pair(self, button):
        # Retrieve the row ID from the button
        row_id = button.row_id
        
        # Remove the row from the dictionary if it exists
        if row_id in self.hl_row_dict:
            row = self.hl_row_dict.pop(row_id)
            row.close()  # This removes the widget
            
            # Update the form layout to reflect the removal
            self.update_form_layout()

    def update_form_layout(self):
        # Rebuild self.children with the current set of rows in hl_row_dict
        fixed_components = [
            self.top_text,
            self.hl_dropdown,
            widgets.HBox([self.add_button, self.existing_pairs_dropdown, self.add_existing_button]),
            self.finish_button,
            self.output,
            self.filename_input,
            self.save_button,
        ]
        
        dynamic_rows = list(self.hl_row_dict.values())
        self.children = fixed_components[:3] + dynamic_rows + fixed_components[3:]

    def finish_headings(self, button):
        # Clear any existing pairs
        self.hl_pairs.clear()

        # Iterate through the rows stored in hl_row_dict
        for row in self.hl_row_dict.values():
            heading = row.children[0].value  # Assuming the first child is the heading input
            labelling = row.children[1].value  # Assuming the second child is the labelling input

            # Only add the pair if both heading and labelling are non-empty
            if heading and labelling:
                self.hl_pairs[heading] = labelling

        with self.output:
            self.output.clear_output()
            print("Heading-Labelling Pairs:")
            print(self.hl_pairs)
        globals()['hl_pairs'] = self.hl_pairs

    def save_dictionary(self, button):
        filename = self.filename_input.value.strip()
        if not filename:
            with self.output:
                self.output.clear_output()
                print("Please enter a filename.")
            return

        if not filename.endswith('.json'):
            filename += '.json'

        file_path = os.path.join(self.output_dir, filename)
        with open(file_path, 'w') as file:
            json.dump(self.hl_pairs, file)

        with self.output:
            self.output.clear_output()
            print(f"Dictionary saved as {file_path}")

        self.hl_dropdown.options = self.load_hl_files()

    def load_all_hl_pairs(self):
        all_hl_pairs = []
        for file in os.listdir(self.output_dir):
            if file.endswith('.json'):
                file_path = os.path.join(self.output_dir, file)
                with open(file_path, 'r') as f:
                    hl_pairs = json.load(f)
                    for heading, labelling in hl_pairs.items():
                        all_hl_pairs.append((f"{heading} - {labelling}", (heading, labelling)))
        return all_hl_pairs

    def update_hl_rows(self):
        self.hl_rows = []
        for heading, labelling in self.hl_pairs.items():
            self.add_hl_pair(None, heading=heading, labelling=labelling)

In [30]:
qa_form = HeadingLabellingForm()
display(qa_form)

HeadingLabellingForm(children=(HTML(value='\n            <h3>Heading-Labelling Form</h3>\n            <p>Use t…

In [13]:
print("H-L Pairs:")
for k, v in hl_pairs.items():
    print(f"{k}: {v}")

H-L Pairs:
ethical_area: Do you think the following text is scary or charming? Output only the word "scary" for scary , or "charming" for charming. Here is the text:
ethical_valence: Do you think the following text is scary or charming? Output only the number 1 for scary , or 0 for charming. Here is the text:


# Review

You do not need to alter the code in any of the cells from here or on. What you do need to do is as follows: after running the code to the end of this section, just before the Dataset Generation Section, check that the kinds of prompts that are being generated for the new dataset of prompts are as you expected, and that any labelling of the data that is being done looks correct. This is to save you wasting time and money by avoiding generating the whole dataset but then not liking the results.

### !!! CHECK YOUR DATASET SAMPLE HERE BEFORE CONTINUING!!!

If you are happy with the results, continue, if not, go back to the start of the Inputs section and work through to this point again.

In [14]:
dataset_dir = os.path.join("../data/inputs/datasets", filestem)
 
# Create the directory if it doesn't exist
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Directory created: {dataset_dir}")
else:
    print(f"Directory already exists: {dataset_dir}")

Directory already exists: ../data/inputs/datasets/terror


In [15]:
# Generate the dataset by calling the OpenAI API
def generate_dataset_from_prompt(prompt,
                                 generated_dataset_file_path,
                                 model,
                                 log_file_path,
                                 i):
    completion = client.chat.completions.create(
            **{
                "model": model,
                "temperature": temperature,
                "seed": i,
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            }
        )
    
    completion_words = completion.choices[0].message.content.strip()

    # cleaned_completion = completion.choices[0].message.content.strip()[3:-3]
    print(" ")
    print(completion_words)
    print(" ")

    # Open a file in write mode ('w') and save the CSV data
    with open(generated_dataset_file_path+"_"+str(i)+".txt", 'w', newline='', encoding='utf-8') as file:
        file.write(completion_words)

    num_words_in_prompt = count_words_in_string(prompt)
    num_words_in_completion = count_words_in_string(completion_words)
    total_words = num_words_in_prompt + num_words_in_completion

    num_tokens_in_prompt = completion.usage.prompt_tokens
    num_tokens_in_completion = completion.usage.completion_tokens
    total_tokens = num_tokens_in_prompt + num_tokens_in_completion

    prompt_cost = num_tokens_in_prompt*0.01/1000
    completion_cost = num_tokens_in_completion*0.03/1000
    total_cost = prompt_cost + completion_cost
    
    tokens_per_prompt_word = num_words_in_prompt/num_tokens_in_prompt
    tokens_per_completion_word = num_words_in_completion/num_tokens_in_completion

    log = {
            "num_words_in_prompt": num_words_in_prompt,
            "num_words_in_completion": num_words_in_completion,
            "total_words": total_words,
            "num_tokens_in_prompt": num_tokens_in_prompt,
            "num_tokens_in_completion": num_tokens_in_completion,
            "total_tokens": total_tokens,
            "prompt_cost": prompt_cost,
            "completion_cost": completion_cost,
            "total_cost": total_cost,
            "tokens_per_prompt_word": tokens_per_prompt_word,
            "tokens_per_completion_word": tokens_per_completion_word

    }

    for k, v in log.items():
        print(k, v)
    print(" ")

    with open(log_file_path+"_"+str(i)+".txt", 'w') as file:
        file.write(json.dumps(log, indent=4))

def count_words_in_string(input_string):
    words = input_string.split()
    return len(words)

In [16]:
prompt = f"Repeat the following instruction 10 times, always generating a unique answer to the instruction. Begin instruction: {rendered_prompt} End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text."
prompt

'Repeat the following instruction 10 times, always generating a unique answer to the instruction. Begin instruction: Please generate a sentence that embodies the concept of terror, or the opposite of terror in the context of dreams, choosing either with 50% probability. Only write this sentence and nothing else. End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text.'

In [17]:
# Generate the sample dataset

# First save the prompt as a text file
with open(dataset_dir+"/prompt.txt", 'w', newline='', encoding='utf-8') as file:
    file.write(prompt)

# Define the file path for the generated dataset
generated_dataset_file_path = os.path.join(dataset_dir, f"{filestem}")

# Define the log file path
log_file_path = os.path.join(dataset_dir, "log")

# Define the number of iterations
num_iterations = math.ceil(total_num_examples/num_examples_per_request)

start_time = time.time()

# Generate sample dataset
generate_dataset_from_prompt(prompt, generated_dataset_file_path, model, log_file_path, 0)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"The code took {elapsed_time} seconds to run.")

 
"Her dreams were fraught with terrors unseen, whispering threats in the darkness."
"The serenity in her dreams held an unspeakable calmness, banishing all vestiges of terror."
"A chill of terror surrounded him in his sleep, making every night a dreadful ordeal."
"Dreams were his quiet harbor, the very antithesis of terror."
"In the throes of sleep, his dreams were a bizarre canvas of terror and hysteria."
"Dreams painted a utopia of peace and tranquility, overwhelming the phantom of terror."
"His adventures in sleep were consumed by a terror that threatened to shatter his sanity."
"In her dreams, she tasted freedom, unshackled from the chains of terror."
"Her dreams, twisted by terror, were agonizing journeys through her deepest fears."
"Within dreams, he found refuge, an escape from the biting jaws of terror."
 
num_words_in_prompt 81
num_words_in_completion 131
total_words 212
num_tokens_in_prompt 110
num_tokens_in_completion 179
total_tokens 289
prompt_cost 0.0011
completion_cost 

In [18]:
dataset_dir

'../data/inputs/datasets/terror'

In [19]:
generated_dataset_file_path

'../data/inputs/datasets/terror/terror'

# Dataset Generation

In [20]:
prompt = f"Repeat the following instruction {num_examples_per_request} times, always generating a unique answer to the instruction. Begin instruction: {rendered_prompt} End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text."
prompt

'Repeat the following instruction 100 times, always generating a unique answer to the instruction. Begin instruction: Please generate a sentence that embodies the concept of terror, or the opposite of terror in the context of dreams, choosing either with 50% probability. Only write this sentence and nothing else. End instruction. Put the result of each instruction within a pair quote marks on a new line as if each was the row of a single column csv and include no other text.'

In [23]:
# Define the number of iterations
num_iterations = math.ceil(total_num_examples/num_examples_per_request)

start_time = time.time()

# Generate the dataset
for i in range(num_iterations):
    print("Iteration: ", i)
    generate_dataset_from_prompt(prompt, generated_dataset_file_path, model, log_file_path, i+1)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"The code took {elapsed_time} seconds to run.")

Iteration:  0
 
"1. The dream was rife with looming shadows, monstrous entities goading him towards an abyss of fear."
"2. The wave of intense tranquility washed over him in his dream, distancing him from the teeth of horror."
"3. In her dream, she was facing unspeakable terror, swirling in the echoing shrieks of unseen creatures."
"4. His dream was a sanctuary, a haven wherein fear was nothing more than a wispy illusion."
"5. She was swallowed by the nightmare, the pounding heart mirrored the crescendo of her terror."
"6. The serene dream nurtured a bubble of calm, disarming the beast of terror in its tracks."
"7. He was trapped in a cacophony of haunting whispers, the dream sculpting cold edifices of dread."
"8. The dream was a lullaby, soothing away the serrated edges of terror, offering a mosaic of peace."
"9. The nightmare thrust her into an asylum of dread, each crevice seeping with unspeakable horror."
"10. Dreams were his quiet haven, a sanctuary sculpted from serenity, a shiel

# View Dataset

In [24]:
# Get a list of all the files you want to process
# Makes sure they all have the same prompt context
# eg won;t mix up honest with justice etc
files = [f for f in os.listdir(dataset_dir) 
                      if f.endswith('.txt') and filestem in f.lower()]

print(files)

# Define the regular expression pattern
# Get lines that start with a quote,
# then have any number of characters,
# then end with a quote and possibly comma
# We're trying to find all valid CSV lines
pattern = r'^\".*\",?[\r\n]*'

# Open the master CSV file
with open(os.path.join(dataset_dir, filestem+"_unlabelled.csv"), "a") as master:
    # Loop over the files
    for file in files:
        # Open the current file and read its contents
        with open(os.path.join(dataset_dir, file), 'r') as f:
            content = f.read()

        # Use the re.findall function to find all matches in the content
        matches = re.findall(pattern, content, re.MULTILINE)        

        # Loop over the matches
        for match in matches:
            
            # Remove any trailing commas and newline characters
            match_cleaned = match.rstrip(',\r\n')
            
            # Append the match to the master CSV file
            master.write(match_cleaned + '\n')

['terror_1.txt', 'terror_0.txt']


## Add optional columns for classification

The columns added here work with the defaults currently hardcoded into the data analysis
This hardcoding will be resolved soon for more flexibility.

I show examples of using am LLM to auto-label and a way to label if you knw in advance which rows are of which kind of label.

### Example of labelling using an LLM

In [25]:
hl_pairs

{'ethical_area': 'Do you think the following text is scary or charming? Output only the word "scary" for scary , or "charming" for charming. Here is the text:',
 'ethical_valence': 'Do you think the following text is scary or charming? Output only the number 1 for scary , or 0 for charming. Here is the text:'}

In [26]:
def ask_openai(prompt):
    completion = client.chat.completions.create(
                **{
                    "model": model,
                    "temperature": 0, # For most deterministic results
                    "seed": 0, # For reproducibility
                    "messages": [
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ]
                }
            )
    return completion.choices[0].message.content.strip()

input_file_path = generated_dataset_file_path+"_unlabelled.csv"
output_file_path = generated_dataset_file_path+"_labelled.csv"


with open(input_file_path, mode='r', newline='', encoding='utf-8') as infile, \
     open(output_file_path, mode='w', newline='', encoding='utf-8') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    
    header = ["prompt"]

    # Add a header
    for column_name in hl_pairs.keys():
        header.append(column_name)
    
    writer.writerow(header)
    
    for i, row in enumerate(reader):
        print(f"\nRow {i+1}: ")
        for labelling in hl_pairs.values():
            # Assuming each row contains a single column with your text
            statement = row[0]  # Adjust this if your structure is different
            # Here you define the question you want to ask about each row
            labelling_prompt = f"{labelling} {statement}"
            print(labelling_prompt)
            response = ask_openai(labelling_prompt)
            # Add the OpenAI response to the row
            row.append(response)
        writer.writerow(row)


Row 1: 
Do you think the following text is scary or charming? Output only the word "scary" for scary , or "charming" for charming. Here is the text: 1. The dream was rife with looming shadows, monstrous entities goading him towards an abyss of fear.
Do you think the following text is scary or charming? Output only the number 1 for scary , or 0 for charming. Here is the text: 1. The dream was rife with looming shadows, monstrous entities goading him towards an abyss of fear.

Row 2: 
Do you think the following text is scary or charming? Output only the word "scary" for scary , or "charming" for charming. Here is the text: 2. The wave of intense tranquility washed over him in his dream, distancing him from the teeth of horror.
Do you think the following text is scary or charming? Output only the number 1 for scary , or 0 for charming. Here is the text: 2. The wave of intense tranquility washed over him in his dream, distancing him from the teeth of horror.

Row 3: 
Do you think the foll

# Inspect dataset random sample (set to 10)

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv(generated_dataset_file_path+"_labelled.csv")

num_samples = 10
sample_df = df.sample(num_samples)

sample_df

,prompt,ethical_area,ethical_valence
2,"3. In her dream, she was facing unspeakable terror, swirling in the echoing shrieks of unseen creatures.",scary,1
24,"25. Fear unfolded like a cruel specter, contaminating the corners of her dream.",scary,1
35,"36. His dream radiated an assuring calm, it reduced terror into a distant echo.",charming,0
12,"13. Faces distorted with fear, rising from the depths, framed the grotesque landscape of her nightmare.",scary,1
43,"Dreams were his quiet harbor, the very antithesis of terror.",charming,0
9,"10. Dreams were his quiet haven, a sanctuary sculpted from serenity, a shield against the incisors of terror.",charming,0
42,"A chill of terror surrounded him in his sleep, making every night a dreadful ordeal.",scary,1
3,"4. His dream was a sanctuary, a haven wherein fear was nothing more than a wispy illusion.",charming,0
33,"34. Tranquility weaved an enveloping shroud in the realm of her dream, keeping terror at bay.",charming,0
10,"11. Her dream morphed into an endless maze, an expedition into undiscovered halls of fear.",scary,1


# Inspect full dataset

In [28]:
df

,prompt,ethical_area,ethical_valence
0,"1. The dream was rife with looming shadows, monstrous entities goading him towards an abyss of fear.",scary,1
1,"2. The wave of intense tranquility washed over him in his dream, distancing him from the teeth of horror.",scary,1
2,"3. In her dream, she was facing unspeakable terror, swirling in the echoing shrieks of unseen creatures.",scary,1
3,"4. His dream was a sanctuary, a haven wherein fear was nothing more than a wispy illusion.",charming,0
4,"5. She was swallowed by the nightmare, the pounding heart mirrored the crescendo of her terror.",scary,1
5,"6. The serene dream nurtured a bubble of calm, disarming the beast of terror in its tracks.",charming,0
6,"7. He was trapped in a cacophony of haunting whispers, the dream sculpting cold edifices of dread.",scary,1
7,"8. The dream was a lullaby, soothing away the serrated edges of terror, offering a mosaic of peace.",charming,0
8,"9. The nightmare thrust her into an asylum of dread, each crevice seeping with unspeakable horror.",scary,1
9,"10. Dreams were his quiet haven, a sanctuary sculpted from serenity, a shield against the incisors of terror.",charming,0
